In [1]:
import os
from sentence_transformers import SentenceTransformer
import gzip
import json
import torch

In [3]:
# Load sentence embeddings model
model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v4_MiniLM-L6', cache_folder='../pretrained')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/mnt/geogpt-gpfs/llm-course/home/jiaminghui/Miniconda3/envs/llm/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# Load astronomy or physics titles generated by gpt4
with open("../datasets/sun-related-wikititle.txt", 'r') as f:
    astro_title_list = f.read().splitlines()

In [59]:
# Load dolma_v1_7 .json.gz
root = "mnt/geogpt-gpfs/llm-course/public/datasets/wikipedia/20231101.en/"
file = "train-00000-of-00041.parquet"
file = os.path.join(root, file)

title_list = []
id_list = []
text_list = []
with gzip.open(file, 'rt') as f:
    for line in f:
        doc = json.loads(line)
        title_list.append(doc['metadata']['title'])
        id_list.append(doc['id'])
        text_list.append(doc['text'])
print(len(title_list))

1084431


In [53]:
# Embedding astronomy titles
target_embeddings = model.encode(astro_title_list, show_progress_bar=True, batch_size=1024, convert_to_numpy=False)
target_embeddings = torch.vstack(target_embeddings)
target_embeddings.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([286, 384])

In [60]:
# Embedding dolma titles
source_embeddings = model.encode(title_list, show_progress_bar=True, batch_size=1024, convert_to_numpy=False, device=torch.device("cuda:7"), normalize_embeddings=True)
source_embeddings = torch.vstack(source_embeddings)
source_embeddings.shape

Batches:   0%|          | 0/1060 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Calculate the similarity matrix of astronomy titles and dolma titles.
# Columns of the matrix represents dolma titles, and the rows represent astronomy titles.
device=torch.device("cuda:7")
title_similar_matrix = torch.matmul(target_embeddings.to(device), source_embeddings.T)
print(title_similar_matrix.shape)

source_max_similar = title_similar_matrix.max(dim=0).values

torch.Size([286, 1721599])


In [ ]:
threshold = 0.8
idx_over_threshold = torch.where(source_max_similar >= threshold)[0].cpu().numpy()

# for idx in idx_over_threshold:
#     print(f"dolma text: {text_list[idx]}")
#     print(f"Dolma title: {title_list[idx]}")
#     print(f"Dolma id: {id_list[idx]}")
#     print("")

In [ ]:
idx_over_threshold.shape

(14,)

In [ ]:
# Save to jsonl file
with open("../datasets/astro_text.jsonl", 'a') as f:
    for idx in idx_over_threshold:
        f.write(json.dumps({
            'id': id_list[idx],
            'source': file,
            'text': text_list[idx]
        }) + '\n')